In [2]:
import os
import tqdm
import pandas as pd

In [3]:
docFolderDir = './S04_AnswerScoring/'
lsDocDir = [f for f in os.listdir(docFolderDir)]
lsDocDir

['Industrial_Gas_Turbines_AMY_Razak',
 'msd_servo_drive',
 'pcs7_compendium_part_a_en-US_en-US',
 'pcs7_compendium_part_b_en-US_en-US']

In [ ]:
lsALL = []

for i in range(len(lsDocDir)):
    docfolder = lsDocDir[i]
    print(docfolder)
    # LIST ALL CSV IN FOLDER
    lscsv = [f for f in os.listdir(f"./S04_AnswerScoring/{docfolder}") if f.endswith('.csv')]
    # LOAD INTO BIG DF
    lsdf = []
    for j in tqdm.tqdm(range(len(lscsv))):
        csvname = lscsv[j]
        df = pd.read_csv(f"./S04_AnswerScoring/{docfolder}/{csvname}")
        if 'Unnamed: 0' in df.columns:
            df = df.drop('Unnamed: 0', axis=1)
        lsdf.append(df)
    # CONCAT
    dfANS = pd.concat(lsdf)
    # WEIGHTED SCORE
    dfANS['WEIGHTED_SCORE'] = (0.3*dfANS['helpfulness']) + (0.3*dfANS['correctness']) + (0.2*dfANS['coherence']) + (0.1*dfANS['complexity']) + (-0.1*dfANS['verbosity'])
    # MARK BEST SCORE
    dfANS['MAX'] = dfANS.groupby(['NODE_HASH', 'Q'])['WEIGHTED_SCORE'].transform(lambda x: x == x.max()).replace({True: 'Y', False: ''})
    # FILTER ONLY BEST
    dfANS = dfANS[dfANS['MAX'] == 'Y']
    # SAVE
    dfANS.to_csv(f"./S05_SelectGoodScore/{docfolder}.csv", index=False)
    lsALL.append(dfANS)

# CONCAT ALL
dfALLALL = pd.concat(lsALL)
dfALLALL['INPUT'] = dfALLALL['Q']
dfALLALL['OUTPUT'] = dfALLALL['A']
dfALLALL = dfALLALL[['INPUT', 'OUTPUT']]

# SAVE
dfALLALL.to_csv(f"./S05_SelectGoodScore/ALLALL.csv", index=False)

Industrial_Gas_Turbines_AMY_Razak


100%|██████████| 381/381 [00:01<00:00, 281.51it/s]


msd_servo_drive


 73%|███████▎  | 182/250 [00:00<00:00, 341.30it/s]